In [ ]:
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv
import json
from IPython.display import display, Markdown
from pydantic import BaseModel
load_dotenv()

In [ ]:
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
client = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
class Queries(BaseModel):
    wide_queries : list[str]
    deep_queries : list[str]

def generate_queries(topic, width, depth, deepdive_topic):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"You are an expert in topic:{topic}. Your task is to generate search queries to aid the user's research on said topic. The user will provide you research width and depth. Width indicates how wide the research needs to be. Depth indicates how deep the research needs to be for a specific topic. You need to generate {width} search queries to cover the width of the research and {depth} search queries to go deeper into the subtopic: {deepdive_topic}.",
        config={
            'response_mime_type':'application/json',
            'response_schema':list[Queries],
        }
    )
    return json.loads(response.text)

In [ ]:
def generate_context(search_query:str):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f"{search_query}",
        config=types.GenerateContentConfig(
            tools=[types.Tool(
                google_search=types.GoogleSearchRetrieval
            )]
        )
    )
    return response.text, response.candidates[0].grounding_metadata.grounding_chunks

In [ ]:
def generate_report():
    """Main function for generating a report."""
    topic = input("Enter the topic: ")
    width = input("Enter the width: ")
    depth = input("Enter the depth: ")
    deepdive_topic = input("Enter the topic you want to dive deep into: ")
    
    sys_instruct = f"You are an expert analyst in the topic: {topic}. You've been given a lot of context (which you produced earlier) here supporting the user's research on said topic. With this information, generate a detailed (1000 words) report as instructed. Be sure to include all sources, persons, objects etc in the report. Additionally, you must dive deeper into {deepdive_topic} as that is what the user would like to dive deeper into. You MUST include a references section and appropriately add citations. Use the Citations object provided to you, each citation has a title and a URI, the citations in the report MUST be hyperlinked with the corresponding uri so that the user can follow it if necessary. Ensure that the report itself adheres to the user's requirements and does not deviate away from the research's goals. Feel free to provide tables if required."
    
    search_queries = generate_queries(topic, width, depth, deepdive_topic)
    total_context = []
    sources = []
    
    for query_type in ["wide_queries", "deep_queries"]:
        for query in search_queries[0][query_type]:
            context, source_list = generate_context(query)
            total_context.append(context)
            for src in source_list:
                print("Retrieved context from ", src.web.title)
                sources.append({"title": src.web.title, "uri": src.web.uri})
                
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        config=types.GenerateContentConfig(
            system_instruction=sys_instruct
        ),
        contents=f"Context: {json.dumps(total_context)} Citations: {json.dumps(sources)}"
    )
    return response.text


In [ ]:
if __name__ == "__main__":
    final_report = generate_report()
    display(Markdown(final_report))